<center><a href="https://sites.google.com/fat.uerj.br/livia/"> <img src="../images/capa2.png" alt="Header" style="width: 800px;"/> </a></center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vnikoofard/DeepLearningTF/blob/main/notebooks/06_headline_generator.ipynb)

# Dados Sequenciais

Neste tutorial, faremos um desvio de dados autônomos, como imagens estáticas, para dados que dependem de outros itens de dados em uma sequência. Para o nosso exemplo, usaremos frases de texto. A linguagem é naturalmente composta de dados sequenciais, na forma de caracteres em palavras e palavras em sentenças. Outros exemplos de dados de sequência incluem preços de ações e dados meteorológicos ao longo do tempo. Os vídeos, embora contenham imagens estáticas, também são sequências. Os elementos dos dados têm relação com o que vem antes e o que vem depois, e esse fato requer uma abordagem diferente.

## Objetivos

* Preparar dados de sequência para usar em uma [rede neural recorrente](https://developers.google.com/machine-learning/glossary#recurrent-neural-network) (RNN)
* Construa e treine um modelo para realizar a previsão de palavras

## Gerador de Manchetes

Todos nós já vimos preditores de texto em aplicativos como as barras de pesquisa, em telefones celulares ou em editores de texto que fornecem preenchimento automático de frases. Muitos dos bons modelos preditores de texto são treinados em conjuntos de dados muito grandes e levam muito tempo e/ou poder de processamento para treinar. Para este exercício, nosso preditor será bastante simples, mas fornecerá uma exposição simples ao processamento de linguagem, dados de sequência e um dos elementos de arquitetura clássica usados para treinar sequências, *redes neurais recorrentes* ou *RNNs*.

## Lendo os dados

Nosso conjunto de dados consiste em manchetes do jornal [New York Times](https://www.nytimes.com/) ao longo de vários meses. Começaremos lendo todas as manchetes dos artigos. Os artigos estão em arquivos CSV, então podemos usar *pandas* para lê-los.

In [ ]:
import os 
import pandas as pd

nyt_dir = 'data/nyt_dataset/articles/'

all_headlines = []
for filename in os.listdir(nyt_dir):
    if 'Articles' in filename:
        # Read in all the data from the CSV file
        headlines_df = pd.read_csv(nyt_dir + filename)
        # Add all of the headlines to our list
        all_headlines.extend(list(headlines_df.headline.values))
len(all_headlines)

Vamos dar uma olhada em nossas primeiras manchetes:

In [ ]:
all_headlines[:20]

## Limpando os dados

Uma parte importante das tarefas de processamento de linguagem natural (NLP) (onde os computadores lidam com a linguagem) é o processamento de texto de forma que os computadores possam entendê-lo. Vamos pegar cada uma das palavras que aparecem em nosso conjunto de dados e representá-las com um número. Isso fará parte de um processo chamado *tokenização*.

Antes de fazermos isso, precisamos ter certeza de que temos bons dados. Algumas manchetes estão listadas como "Desconhecidas". Não queremos esses itens em nosso conjunto de treinamento, então vamos filtrá-los:

In [ ]:
# Remova todos os manchetes com o valor "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

Vamos dar outra olhada:

In [ ]:
all_headlines[:20]

Também queremos remover a pontuação e tornar nossas sentenças todas minúsculas, porque isso tornará nosso modelo mais fácil de treinar. Para nossos propósitos, há pouca ou nenhuma diferença entre uma linha que termina com "!" ou "?" ou se as palavras são maiúsculas, como em "The" ou minúsculas, como em "the". Com menos tokens exclusivos, nosso modelo será mais fácil de treinar.

Poderíamos filtrar nossas sentenças antes da tokenização, mas não precisamos porque tudo isso pode ser feito usando o Keras `Tokenizer`.

## Tokenização (Tokenization)

No momento, nosso conjunto de dados consiste em um conjunto de títulos, cada um composto por uma série de palavras. Queremos dar ao nosso modelo uma maneira de representar essas palavras de uma forma que ele possa entender. Com a tokenização, separamos um pedaço de texto em pedaços menores (tokens), que neste caso são palavras. Cada palavra única recebe um número, pois essa é uma maneira de nosso modelo entender os dados. Keras tem uma classe que nos ajudará a tokenizar nossos dados:

```python
tf.keras.preprocessing.text.Tokenizer(
    num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0, **kwargs
)
```

Dando uma olhada na classe [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) no Keras, vemos que os valores padrão já estão configurados para o nosso caso de uso . A string `filters` já remove a pontuação e o sinalizador `lower` define as palavras para letras minúsculas.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize as palavras em nossos manchetes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Podemos dar uma olhada rápida no dicionário `word_index` para ver como o tokenizer salva as palavras:

In [ ]:
# Imprime um subconjunto do dicionário word_index criado pelo Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

Podemos usar o método `texts_to_sequences` para ver como o tokenizer salva as palavras:

In [ ]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

## Criando Sequências

Agora que já tokenizamos os dados, transformando cada palavra em um número representativo, vamos criar sequências de tokens a partir dos títulos. Essas sequências são o que vamos treinar nosso modelo de aprendizado profundo.

Por exemplo, vamos pegar a manchete, "nvidia lança gpus de rastreamento de raios". Cada palavra será substituída por um número correspondente, por exemplo: nvidia - 5, launch - 22, ray - 94, tracing - 16, gpus - 102. A sequência completa seria: [5, 22, 94, 16, 102]. No entanto, também é valioso treinar nas sequências menores do título, como "lançamentos da nvidia". Pegaremos cada título e criaremos um conjunto de sequências para preencher nosso conjunto de dados. Em seguida, vamos usar nosso tokenizador para converter nossos títulos em um conjunto de sequências.

In [ ]:
# Convert data to sequence of tokens 
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

## Padding as Sequencias

No momento, nossas sequências têm vários comprimentos. Para que nosso modelo possa treinar com os dados, precisamos fazer com que todas as sequências tenham o mesmo comprimento. Para fazer isso, adicionaremos preenchimento (*padding*) às sequências. O Keras possui um [método] `pad_sequences` integrado (https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) que podemos usar.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

## Criando Preditores e Rotulos

Também queremos dividir nossas sequências em preditores e um destino. A última palavra da sequência será nosso rotulo e as primeiras palavras da sequência serão nossos preditores. Como exemplo, dê uma olhada na manchete completa: "nvidia lança placas gráficas de ampère"

<table>
<tr><td>PREDITOR </td> <td>           ROTULO </td></tr>
<tr><td>nvidia                   </td> <td>  releases </td></tr>
<tr><td>nvidia releases               </td> <td>  ampere </td></tr>
<tr><td>nvidia releases ampere      </td> <td>  graphics</td></tr>
<tr><td>nvidia releases ampere graphics </td> <td>  cards</td></tr>
</table>

In [ ]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

Como nossas seções anteriores, esses rotulos são categóricas. Estamos prevendo uma palavra de nosso vocabulário total possível. Em vez da rede prever números escalares, faremos com que ela preveja categorias binárias.

In [ ]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

## Criando o modelo

Para nosso modelo, vamos usar algumas novas camadas para lidar com nossos dados sequenciais.

### [Embedding Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

Nossa primeira camada é uma camada de incorporação:

```Python
model.add(Embedding(input_dimension, output_dimension, input_length=input_len))
```

Essa camada receberá as sequências tokenizadas e aprenderá uma incorporação para todas as palavras no conjunto de dados de treinamento. Matematicamente, os embeddings funcionam da mesma forma que um neurônio em uma rede neural, mas conceitualmente, seu objetivo é reduzir o número de dimensões para alguns ou todos os recursos. Nesse caso, ele representará cada palavra como um vetor e as informações contidas nesse vetor conterão as relações entre cada palavra.

Saiba mais sobre a incorporação de camadas [aqui](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/).

<img src="./images/embedding.png" style="largura: 300px;">

### [Long Short Term Memory Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

Nossa próxima camada, e muito importante, é uma camada de memória de longo prazo (LSTM). Um LSTM é um tipo de rede neural recorrente ou RNN. Ao contrário das redes feedforward tradicionais que vimos até agora, as redes recorrentes possuem loops, permitindo que as informações persistam. Aqui está uma representação de uma rede recorrente:

<img src="../images/rnn_rolled.png" style="width: 150px;">

Novas informações (x) são passadas para a rede, que gera uma previsão (h). Além disso, as informações dessa camada são salvas e usadas como entrada para a próxima previsão. Isso pode parecer um pouco complicado, mas vamos dar uma olhada no desenrolar:

<img src="../images/rnn_unrolled.png" style="width: 600px;">

Podemos ver que quando um novo dado (x) é inserido na rede, essa rede emite uma previsão (h) e também passa algumas informações para a próxima camada. Essa próxima camada obtém outro dado, mas também aprende com a camada anterior.

As RNNs tradicionais sofrem com o problema de informações mais recentes contribuírem mais do que informações mais antigas. LSTMs são um tipo especial de camada recorrente capaz de aprender e reter informações de longo prazo. Se você quiser ler mais sobre RNNs e LSTMs, recomendamos [este artigo](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

Bom, vamos criar nosso modelo:

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1 

model = Sequential()

# Add input embedding layer
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.summary()

## Compilando o Modelo

Como antes, compilamos nosso modelo com entropia cruzada categórica, já que prevemos categoricamente uma palavra de nosso vocabulário total. Nesse caso, não vamos usar a precisão como métrica, porque a previsão de texto não é medida como sendo mais ou menos precisa da mesma forma que a classificação de imagens.

Também vamos selecionar um otimizador específico que seja adequado para tarefas LSTM, chamado de otimizador *Adam*. Você pode ler mais sobre os otimizadores, incluindo o otimizador Adam [aqui](https://medium.com/datadriveninvestor/overview-of-different-optimizers-for-neural-networks-e0ed119440c3).

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Treinando o Modelo

Semelhante às seções anteriores, ajustamos nosso modelo da mesma maneira. Desta vez vamos treinar por 30 épocas, o que levará alguns minutos. Você notará que não temos uma pontuação de precisão de treinamento ou validação neste caso. Isso reflete nosso problema diferente de predição de texto.

In [ ]:
model.fit(predictors, labels, epochs=30, verbose=1)

## Discussão dos Resultados

Podemos ver que a perda diminuiu ao longo do treinamento. Poderíamos treinar nosso modelo ainda mais para diminuir a perda, mas isso levaria algum tempo, e não estamos procurando um preditor de texto perfeito no momento. Em seguida, vamos tentar usar o modelo para fazer previsões.

## Fazendo previsões

Para fazer previsões, precisamos começar com um texto inicial e prepará-lo da mesma forma que preparamos nosso conjunto de dados. Isso significa tokenização e preenchimento. Depois de fazer isso, podemos passá-lo para o nosso modelo para fazer uma previsão. Vamos criar uma função para fazer isso chamada `predict_next_token`:

In [ ]:
def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    prediction = model.predict_classes(token_list, verbose=0)
    return prediction

In [ ]:
prediction = predict_next_token("today in new york")
prediction

Vamos usar nosso tokenizador para decodificar a palavra prevista:

In [ ]:
tokenizer.sequences_to_texts([prediction])

## Gerar novos títulos

Agora que podemos prever novas palavras, vamos criar uma função que pode prever manchetes com mais de uma palavra. A função abaixo cria um novo título de tamanho arbitrário.

In [ ]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([prediction])[0]
        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    # Return headline as title-case
    return seed_text.title()

Agora tente algumas manchetes!

In [ ]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']
for seed in seed_texts:
    print(generate_headline(seed, next_words=5))

Os resultados podem ser um pouco abaixo do esperado após 30 épocas de treinamento. Podemos notar que a maioria das manchetes faz algum tipo de sentido gramatical, mas não necessariamente indica uma boa compreensão contextual. Os resultados podem melhorar um pouco executando mais épocas. Você pode fazer isso rodando a célula `fit` de treinamento novamente (e novamente!) para treinar outras 30 épocas de cada vez. Você deve ver o valor da perda diminuir. Em seguida, tente os testes novamente. Os resultados podem variar bastante!

Outras melhorias seriam tentar usar incorporações pré-treinadas com Word2Vec ou GloVe, em vez de aprendê-las durante o treinamento, como fizemos com a camada Keras Embedding. Algumas informações sobre como fazer isso podem ser encontradas [aqui](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).

Em última análise, no entanto, o NLP foi além dos modelos LSTM simples para modelos pré-treinados baseados no Transformer, que são capazes de aprender o contexto do idioma a partir de grandes quantidades de dados textuais, como a Wikipedia. Esses modelos pré-treinados são usados como ponto de partida para o aprendizado de transferência para resolver tarefas de PNL, como a que acabamos de tentar para a conclusão do texto. Você mesmo pode experimentar um desses modelos verificando este [previsor de texto de última geração aqui](https://transformer.huggingface.co/doc/gpt2-large) baseado em um [modelo GPT-2] (https://openai.com/blog/better-language-models/).

Para saber mais sobre os modelos baseados no Transformer, confira [este blog](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) em representações de codificador bidirecional de Transformers (BERT) e procure informações sobre cursos adicionais na página "Próximas etapas" para este curso DLI.

## Resumo

Ótimo trabalho! Você treinou com sucesso um modelo para prever palavras em um título e usou esse modelo para criar títulos de vários comprimentos. Sinta-se à vontade para experimentar e gerar mais alguns títulos.

### Resetar o ambiente
Antes de prosseguir, execute a célula a seguir para resetar o ambiente. Isso é necessário para passar para o próximo notebook.

In [ ]:
import os
os._exit(00)

## Next

Parabéns, você concluiu todos os objetivos de aprendizagem do curso!